In [42]:
import pandas as pd
import os 
import random
from tqdm import tqdm
import matplotlib.pyplot as plt
from datetime import datetime

In [3]:
filepath = "data/labelled_data/"

In [4]:
all_files = os.listdir(filepath)
comments = {}
posts = {}
for fname in tqdm(all_files):
    file = pd.read_csv(filepath+fname,sep='\t').drop('Unnamed: 0',axis=1)
    if 'submission' in fname:
        posts['_'.join(fname.split('_')[3:6])] = file      
    if 'comment' in fname:
        comments['_'.join(fname.split('_')[3:6])] = file
        

100%|██████████| 590/590 [00:09<00:00, 62.73it/s]


In [5]:
print(sorted(posts))

['2022_01_02', '2022_01_03', '2022_01_04', '2022_01_05', '2022_01_06', '2022_01_07', '2022_01_08', '2022_01_09', '2022_01_10', '2022_01_11', '2022_01_12', '2022_01_13', '2022_01_14', '2022_01_15', '2022_01_16', '2022_01_17', '2022_01_18', '2022_01_19', '2022_01_20', '2022_01_21', '2022_01_22', '2022_01_23', '2022_01_24', '2022_01_25', '2022_01_26', '2022_01_27', '2022_01_28', '2022_01_29', '2022_01_30', '2022_01_31', '2022_02_01', '2022_02_02', '2022_02_03', '2022_02_04', '2022_02_05', '2022_02_06', '2022_02_07', '2022_02_08', '2022_02_09', '2022_02_10', '2022_02_11', '2022_02_12', '2022_02_13', '2022_02_14', '2022_02_15', '2022_02_16', '2022_02_17', '2022_02_18', '2022_02_19', '2022_02_20', '2022_02_21', '2022_02_22', '2022_02_23', '2022_02_24', '2022_02_25', '2022_02_26', '2022_02_27', '2022_02_28', '2022_03_01', '2022_03_02', '2022_03_03', '2022_03_04', '2022_03_05', '2022_03_06', '2022_03_07', '2022_03_08', '2022_03_09', '2022_03_10', '2022_03_11', '2022_03_12', '2022_03_13', '2022

In [6]:
posts['2022_01_02'].columns

Index(['all_awardings', 'allow_live_comments', 'author',
       'author_flair_background_color', 'author_flair_css_class',
       'author_flair_richtext', 'author_flair_text', 'author_flair_text_color',
       'author_flair_type', 'author_fullname', 'author_is_blocked',
       'author_patreon_flair', 'author_premium', 'awarders', 'can_mod_post',
       'contest_mode', 'created_utc', 'domain', 'full_link', 'gildings', 'id',
       'is_created_from_ads_ui', 'is_crosspostable', 'is_meta',
       'is_original_content', 'is_reddit_media_domain', 'is_robot_indexable',
       'is_self', 'is_video', 'link_flair_background_color',
       'link_flair_richtext', 'link_flair_text_color', 'link_flair_type',
       'locked', 'media_only', 'no_follow', 'num_comments', 'num_crossposts',
       'over_18', 'parent_whitelist_status', 'permalink', 'pinned', 'pwls',
       'retrieved_on', 'score', 'selftext', 'send_replies', 'spoiler',
       'stickied', 'subreddit', 'subreddit_id', 'subreddit_subscribers'

In [8]:
def trash_holded_count(series,trash_hold = 0.5):
    # count number of records with higher than given value in column
    
    count = 0
    for val in series:
        if val > trash_hold:
            count +=1
    
    return count

    

In [9]:
social_dimensions = ['support','knowledge', 'conflict', 'power',
                    'similarity', 'fun', 'status',
                    'trust', 'identity', 'romance']
func_list = ['mean','count',lambda x : trash_holded_count(x, 0.6)]
agg_dict={ dim : func_list for dim in social_dimensions}


df =posts['2022_05_17'].groupby("subreddit").agg( agg_dict)

In [48]:
df.support['mean']['Bitcoin']

0.21377085228450596

In [16]:
coins = ['Bitcoin','XRP','Ethereum','Solana','Polkadot']

In [20]:
from itertools import product,permutations

In [39]:
dim_metrics = ['mean','trash_holded_count']
cols =["_".join(col) for col in list(product(social_dimensions,dim_metrics))]
print(cols)
cols.append('count')
cols= list(product(coins,cols))
#cols= list(zip(cols,coins))

print(cols)

['support_mean', 'support_trash_holded_count', 'knowledge_mean', 'knowledge_trash_holded_count', 'conflict_mean', 'conflict_trash_holded_count', 'power_mean', 'power_trash_holded_count', 'similarity_mean', 'similarity_trash_holded_count', 'fun_mean', 'fun_trash_holded_count', 'status_mean', 'status_trash_holded_count', 'trust_mean', 'trust_trash_holded_count', 'identity_mean', 'identity_trash_holded_count', 'romance_mean', 'romance_trash_holded_count']
[('Bitcoin', 'support_mean'), ('Bitcoin', 'support_trash_holded_count'), ('Bitcoin', 'knowledge_mean'), ('Bitcoin', 'knowledge_trash_holded_count'), ('Bitcoin', 'conflict_mean'), ('Bitcoin', 'conflict_trash_holded_count'), ('Bitcoin', 'power_mean'), ('Bitcoin', 'power_trash_holded_count'), ('Bitcoin', 'similarity_mean'), ('Bitcoin', 'similarity_trash_holded_count'), ('Bitcoin', 'fun_mean'), ('Bitcoin', 'fun_trash_holded_count'), ('Bitcoin', 'status_mean'), ('Bitcoin', 'status_trash_holded_count'), ('Bitcoin', 'trust_mean'), ('Bitcoin', '

In [60]:
aggs_dict = {'_'.join(col): [] for col in cols}
aggs_dict['Date'] = []


In [61]:
# create right data frame
# with seperate col for each coin for each of the measures
# write down coinsXRP
for date in sorted(posts):
    res_df = posts[date].groupby("subreddit").agg( agg_dict)
    current_date = datetime.strptime(date, '%Y_%m_%d')
    aggs_dict['Date'].append(current_date)
    for idx, dim in enumerate(social_dimensions):
        temp_df = res_df[dim]
        temp_df.rename(columns = {'<lambda_0>': 'trash_holded_count'},inplace= True)
        for coin in coins:
            if coin in res_df.index:
                if idx == 0:
                    aggs_dict[f'{coin}_count'].append(temp_df['count'][coin])
                metric1 = 'mean'
                key1 = f'{coin}_{dim}_{metric1}'
                aggs_dict[key1].append(temp_df[metric1][coin])
                metric2 = 'trash_holded_count'
                key2 = f'{coin}_{dim}_{metric2}'
                aggs_dict[key2].append(temp_df[metric2][coin])
            else: 
                if idx == 0:
                    aggs_dict[f'{coin}_count'].append(0)
                metric1 = 'mean'
                key1 = f'{coin}_{dim}_{metric1}'
                aggs_dict[key1].append(0)
                metric2 = 'trash_holded_count'
                key2 = f'{coin}_{dim}_{metric2}'
                aggs_dict[key2].append(0)                




/home/bence/.local/lib/python3.8/site-packages/pandas/core/frame.py:4293: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [63]:
agg_df = pd.DataFrame(aggs_dict)

In [64]:
agg_df

,Bitcoin_support_mean,Bitcoin_support_trash_holded_count,Bitcoin_knowledge_mean,Bitcoin_knowledge_trash_holded_count,Bitcoin_conflict_mean,Bitcoin_conflict_trash_holded_count,Bitcoin_power_mean,Bitcoin_power_trash_holded_count,Bitcoin_similarity_mean,Bitcoin_similarity_trash_holded_count,...,Polkadot_status_mean,Polkadot_status_trash_holded_count,Polkadot_trust_mean,Polkadot_trust_trash_holded_count,Polkadot_identity_mean,Polkadot_identity_trash_holded_count,Polkadot_romance_mean,Polkadot_romance_trash_holded_count,Polkadot_count,Date
0,0.222364,11.0,0.774394,126.0,0.360903,24.0,0.260957,16.0,0.282526,16.0,...,0.374761,1.0,0.436463,2.0,0.424804,0.0,0.125997,0.0,10,2022-01-02
1,0.232385,12.0,0.757553,144.0,0.335469,23.0,0.322267,28.0,0.309141,19.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0,2022-01-03
2,0.238770,12.0,0.762324,136.0,0.358066,20.0,0.297448,20.0,0.326754,24.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0,2022-01-04
3,0.267636,7.0,0.724305,127.0,0.347417,16.0,0.345323,22.0,0.340669,16.0,...,0.345958,0.0,0.325666,4.0,0.432508,0.0,0.152418,1.0,19,2022-01-05
4,0.226952,7.0,0.750199,131.0,0.371424,29.0,0.312506,23.0,0.339098,24.0,...,0.393741,1.0,0.358508,3.0,0.457962,0.0,0.124863,0.0,28,2022-01-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,0.259344,2.0,0.678682,56.0,0.408663,16.0,0.380907,17.0,0.347852,7.0,...,0.302641,0.0,0.188923,0.0,0.442253,0.0,0.017239,0.0,6,2022-11-05
306,0.234359,4.0,0.739649,71.0,0.368906,20.0,0.317771,12.0,0.346554,9.0,...,0.334008,0.0,0.243536,0.0,0.449607,0.0,0.170151,0.0,5,2022-11-06
307,0.310390,7.0,0.683969,55.0,0.355795,10.0,0.357239,11.0,0.335650,6.0,...,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0,2022-11-07
308,0.229478,5.0,0.745658,83.0,0.398035,18.0,0.338302,20.0,0.268616,6.0,...,0.357398,0.0,0.239262,0.0,0.426516,0.0,0.118626,0.0,9,2022-11-08
